# BIP Framework training example 

In [1]:
using Pkg
Pkg.activate("..")
using BIPs

  Activating project at `/z1-josemm/josemm/MyDocs/personal_BIP/BIPs.jl`


In [2]:
using Statistics
using Pkg.Artifacts
using DelimitedFiles


Lets begin by bringing in the dataset. It contains tree splits:
* **train**: the training set with 1M jets
* **validation**: the validation set with 400k jets

And of course later we will use the **test** set with other 400k jets to report the results


In [3]:
dataset_path = "../../../DataLake/raw"
# dataset_path = "/Users/ortner/datasets/toptagging"

train_data_path = dataset_path*"/train.h5"
val_data_path = dataset_path*"/val.h5"

"../../../DataLake/raw/val.h5"

### Reading the data

In order to read the datasets, we call the `read_dataset` function:
to read the TopQuark format

In [4]:
train_jets, train_labels = BIPs.read_data("TQ", train_data_path)
train_labels = [reinterpret(Bool, b == 1.0) for b in train_labels]
print("Number of entries in the training data: ", length(train_jets))

Number of entries in the training data: 1210997

In [5]:
val_jets, val_labels = BIPs.read_data("TQ", val_data_path)
val_labels = [reinterpret(Bool, b == 1.0) for b in val_labels]
print("Number of entries in the validation data: ", length(val_jets))

Number of entries in the validation data: 402999

Lets examine how one of the jets looks like, each one of the entries is one detected particle's four momentum $(E, p_x, p_y, p_z)$.

However,in order to compute the embeddings, it is necesary to convert the jets to a format that can be used by the framework. The function `data2hyp` allows to convert each detected four momentum to the jet basis, a.k.a $(\tilde p_T, \cos(\theta), \sin(\theta), \tilde y, E_T)$

In [6]:
train_transf_jets = data2basis(train_jets; basis="lab")
val_transf_jets = data2basis(val_jets;  basis="lab")
println("Transformed jets")

Transformed jets


### The embeddings

Once the jets are converted to the jet basis, it is moment to embed the model using the *Invariant Polynomials*. 

The function `build_ip` allocates efficiently the sparse basis, while the `bip_data` computes the invariant representation of each one of the jets.

In [7]:
f_bip, specs = build_ip(order=3, levels=6)
    
function bip_data(dataset_jets)
    storage = zeros(length(dataset_jets), length(specs))
    for i = 1:length(dataset_jets)
        storage[i, :] = f_bip(dataset_jets[i])
    end
    storage[:, 2:end]
end

bip_data (generic function with 1 method)

In [8]:
train_embedded_jets = bip_data(train_transf_jets)
println("Embedded train jets correclty")
val_embedded_jets = bip_data(val_transf_jets)
println("Embedded test jets correclty")

Embedded train jets correclty


Embedded test jets correclty


## Test data

In [9]:
test_data_path = "../../../DataLake/raw/test.h5"
test_jets, test_labels = BIPs.read_data("TQ", test_data_path)
test_labels = [reinterpret(Bool, b == 1.0) for b in test_labels]
test_transf_jets = data2basis(test_jets; basis="lab")
test_embedded_jets = bip_data(test_transf_jets)
print("Embedded test jets correclty")

Embedded test jets correclty

In [10]:
scale(A) = (A .- mean(A, dims=1)) ./ std(A, dims=1)

##
X_train, y_train = scale(train_embedded_jets), train_labels
X_test, y_test = scale(test_embedded_jets), test_labels
X_val, y_val = scale(val_embedded_jets), val_labels


([-1.5684881899977778 0.21517065062201138 … 1.0583723074179052 0.9999980443685902; -1.0464540047230453 -1.2129042835958876 … 1.3602321530710515 0.8849767488171001; … ; 0.20940060839779454 -1.256219961289861 … 0.9333375308164582 0.11691180598158092; -0.03474706067294569 1.3757871735615792 … -1.0559815524034395 0.345823366471957], Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# CLASSIFIERS

### MLP

In [11]:
using PyCall
@pyimport sklearn.neural_network as sk_nn

In [12]:
mlp = sk_nn.MLPClassifier(verbose=true, max_iter=2000, hidden_layer_sizes=(50,), n_iter_no_change=50)

PyObject MLPClassifier(hidden_layer_sizes=(50,), max_iter=2000, n_iter_no_change=50,
              verbose=True)

In [13]:
mlp.fit(X_train, train_labels)

Iteration 1, loss = 0.26617843
Iteration 2, loss = 0.23981651
Iteration 3, loss = 0.23579456
Iteration 4, loss = 0.23384599
Iteration 5, loss = 0.23238238
Iteration 6, loss = 0.23154185
Iteration 7, loss = 0.23021646
Iteration 8, loss = 0.22962900
Iteration 9, loss = 0.22926098
Iteration 10, loss = 0.22867599
Iteration 11, loss = 0.22806906
Iteration 12, loss = 0.22778734
Iteration 13, loss = 0.22702670
Iteration 14, loss = 0.22667369
Iteration 15, loss = 0.22623597
Iteration 16, loss = 0.22589829
Iteration 17, loss = 0.22561057
Iteration 18, loss = 0.22543174
Iteration 19, loss = 0.22528023
Iteration 20, loss = 0.22522938
Iteration 21, loss = 0.22482692
Iteration 22, loss = 0.22443516
Iteration 23, loss = 0.22437161
Iteration 24, loss = 0.22415949
Iteration 25, loss = 0.22404283
Iteration 26, loss = 0.22387212
Iteration 27, loss = 0.22376232
Iteration 28, loss = 0.22346804
Iteration 29, loss = 0.22338139
Iteration 30, loss = 0.22334900
Iteration 31, loss = 0.22312686
Iteration 32, los

PyObject MLPClassifier(hidden_layer_sizes=(50,), max_iter=2000, n_iter_no_change=50,
              verbose=True)

In [14]:
mlp.score(X_val, val_labels)

0.9131288166968157

In [15]:
mlp_test_scores = mlp.predict_proba(X_test)
writedlm( "../foo/lab_mlp_test_probas.csv",  mlp_test_scores, ',')


## XGboost

In [16]:
using XGBoost
function fit_xgb_clf(X_train, y_train, X_val, y_val)
    dtrain = DMatrix(X_train, label=y_train)
    dtest = DMatrix(X_val, label=y_val)
    param = ["silent" => 0, "subsample" => 0.5, "colsample_bytree" => 0.5, "eta" => 0.05]
    watchlist = [(dtest, "test"), (dtrain, "train")]
    bst = xgboost(X_train, 500, label=y_train, param=param, objective = "binary:logistic",
        watchlist=watchlist, metrics=["logloss", "auc", "error"],
        early_stopping_rounds=50, verbose_eval=10, seed=137)
    
    bst
end

fit_xgb_clf (generic function with 1 method)

In [17]:
model = fit_xgb_clf(X_train, y_train, X_val, val_labels)

[1]	test-logloss:0.66997270397302366	test-auc:0.88351529804711926	test-error:0.19104017627835304	train-logloss:0.66991185481309434	train-auc:0.88414394392901396	train-error:0.19012103250462223
[2]	test-logloss:0.65016633669275015	test-auc:0.89363516203774329	test-error:0.18348680765957234	train-logloss:0.65003061912279847	train-auc:0.89480034394579533	train-error:0.18226139288536636


[3]	test-logloss:0.63178779897786985	test-auc:0.90201458337532125	test-error:0.18010962806359321	train-logloss:0.63160674549174789	train-auc:0.90304050286213988	train-error:0.17899796613864444
[4]	test-logloss:0.61365271583505110	test-auc:0.90442420403804369	test-error:0.17800540447991187	train-logloss:0.61339257761716948	train-auc:0.90550643117954721	train-error:0.17710365921633167


[5]	test-logloss:0.59857038044368416	test-auc:0.90578615990113698	test-error:0.17576470413077949	train-logloss:0.59825373828219486	train-auc:0.90687601544047447	train-error:0.17477747674023966
[6]	test-logloss:0.58472345621543331	test-auc:0.90913941483654048	test-error:0.17435527135303067	train-logloss:0.58434749304007094	train-auc:0.91034633504975904	train-error:0.17347524395188427


[7]	test-logloss:0.57041051022768885	test-auc:0.90890190908602586	test-error:0.17482425514703512	train-logloss:0.56999176944422403	train-auc:0.91008487975371133	train-error:0.17418209954277344
[8]	test-logloss:0.55665425448502925	test-auc:0.91287289255805004	test-error:0.16886146119469278	train-logloss:0.55619014466035677	train-auc:0.91406775521263861	train-error:0.16819447116714575


[9]	test-logloss:0.54495646703448219	test-auc:0.91372674629907802	test-error:0.16870761465909345	train-logloss:0.54443684666872538	train-auc:0.91499320434293829	train-error:0.16804087871398526
[10]	test-logloss:0.53320182673506922	test-auc:0.91702679428852363	test-error:0.16537261879061735	train-logloss:0.53261862165284513	train-auc:0.91838969507928048	train-error:0.16449669156901298


...

[493]	test-logloss:0.24652303914042631	test-auc:0.95875267137697540	test-error:0.10204491822560353	train-logloss:0.23646840525204382	train-auc:0.96299815742872319	train-error:0.09701592984953721
[494]	test-logloss:0.24649769379337924	test-auc:0.95875824719170388	test-error:0.10203002985119070	train-logloss:0.23642970340898062	train-auc:0.96300708242390931	train-error:0.09699528570260703


[495]	test-logloss:0.24647042376660161	test-auc:0.95876516711349158	test-error:0.10200769728957144	train-logloss:0.23639382528910738	train-auc:0.96301616186429095	train-error:0.09696720966278199
[496]	test-logloss:0.24644210187106025	test-auc:0.95877767618677334	test-error:0.10198536472795218	train-logloss:0.23634282523962397	train-auc:0.96303772029862855	train-error:0.09694656551585182


[497]	test-logloss:0.24642529255367285	test-auc:0.95878086162949849	test-error:0.10200273449810049	train-logloss:0.23631640728594519	train-auc:0.96304331392507758	train-error:0.09693335326181650
[498]	test-logloss:0.24640902847485191	test-auc:0.95878770629646248	test-error:0.10199777170662955	train-logloss:0.23628677705378182	train-auc:0.96305693961269423	train-error:0.09692509560304444


[499]	test-logloss:0.24636790499575201	test-auc:0.95880594711256595	test-error:0.10198536472795218	train-logloss:0.23622673825919008	train-auc:0.96308348828192669	train-error:0.09690362569023706
[500]	test-logloss:0.24634840544345965	test-auc:0.95881188909108539	test-error:0.10195062518765555	train-logloss:0.23619035953524681	train-auc:0.96309465384801785	train-error:0.09686316316225391


Booster(Ptr{Nothing} @0x0000000020a2c150)

In [18]:
xgb_probas = predict(model, X_test)

403997-element Vector{Float32}:
 0.00077848166
 0.14187185
 0.002530643
 0.43887255
 0.0033585855
 0.12748805
 0.7721033
 0.003029308
 0.83401877
 0.00046503125
 ⋮
 0.86056775
 0.75044924
 0.8752506
 0.738889
 0.67851615
 0.3130518
 0.9806183
 0.9518672
 0.8244352

In [19]:
writedlm( "../foo/lab_xgb_test_probas2.csv",  xgb_probas, ',')